Research topics: 
1. Use height & weight to predict players average score
2. Use height, weight, college, position to predict longetivity of players

Initial thoughts:
- Linear / Lasso / Ridge regression
- xgboost

In [124]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.linear_model import LinearRegression
import os

In [125]:
# Import files

folder_path = r'C:\Users\User\Documents\venv\nba player analysis'

#players = pd.read_csv(os.path.join(folder_path, 'Players.csv'))
#season_stats = pd.read_csv(os.path.join(folder_path, 'Seasons_Stats.csv'))

In [126]:
# Check if data contains null / NA values
'''
plt.figure(figsize=(12,7))
plt.subplot(2,1,1)
plt.title('Players')
sn.heatmap(players.isna())

plt.subplot(2,1,2)
plt.title('Stats')
sn.heatmap(season_stats.isna())
'''

"\nplt.figure(figsize=(12,7))\nplt.subplot(2,1,1)\nplt.title('Players')\nsn.heatmap(players.isna())\n\nplt.subplot(2,1,2)\nplt.title('Stats')\nsn.heatmap(season_stats.isna())\n"

In [127]:
season_stats = pd.read_csv(os.path.join(folder_path, 'Seasons_Stats.csv'))

In [128]:
season_stats

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24686,24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,...,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0
24687,24687,2017.0,Tyler Zeller,C,27.0,BOS,51.0,5.0,525.0,13.0,...,0.564,43.0,81.0,124.0,42.0,7.0,21.0,20.0,61.0,178.0
24688,24688,2017.0,Stephen Zimmerman,C,20.0,ORL,19.0,0.0,108.0,7.3,...,0.600,11.0,24.0,35.0,4.0,2.0,5.0,3.0,17.0,23.0
24689,24689,2017.0,Paul Zipser,SF,22.0,CHI,44.0,18.0,843.0,6.9,...,0.775,15.0,110.0,125.0,36.0,15.0,16.0,40.0,78.0,240.0


In [129]:
# Make stats table into points / minute played per player

possible_X = ['Year', 'Player', 'Age', 'Pos', 'Tm']
possible_y = ['DWS', 'PTS', 'PER']

X_and_y = possible_X + possible_y

season_stats.dropna(subset=X_and_y, inplace=True)
season_stats = season_stats[X_and_y]

season_stats[['Pos1','Pos2']] = season_stats['Pos'].str.split('-', 1, expand=True)
season_stats = pd.get_dummies(season_stats, columns=['Pos1'])
season_stats['Year'] = season_stats['Year'].astype(int)

agg_season_stats = season_stats.groupby([
    'Year', 
    'Player',
    'Age',
    'Pos1_C',
    'Pos1_F',
    'Pos1_G',
    'Pos1_PF',
    'Pos1_PG',
    'Pos1_SF',
    'Pos1_SG',
    'DWS',
    'Tm',
    'PTS',
    'PER']).mean().reset_index()

C:\Users\User\AppData\Local\Temp\ipykernel_21032\4145911118.py:11: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  season_stats[['Pos1','Pos2']] = season_stats['Pos'].str.split('-', 1, expand=True)
C:\Users\User\AppData\Local\Temp\ipykernel_21032\4145911118.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_stats[['Pos1','Pos2']] = season_stats['Pos'].str.split('-', 1, expand=True)
C:\Users\User\AppData\Local\Temp\ipykernel_21032\4145911118.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [130]:
# Merge player table & stats table

players = pd.read_csv(os.path.join(folder_path, 'Players.csv'))
agg_season_stats = pd.merge(agg_season_stats, players[['Player', 'height', 'weight', 'collage']], on='Player', how='left')
agg_season_stats[['height', 'weight']] = agg_season_stats[['height', 'weight']].astype(int)
agg_season_stats = pd.get_dummies(agg_season_stats, columns=['collage', 'Tm'])
agg_season_stats.drop(columns=['Player'], inplace=True)

In [131]:
agg_season_stats.isna().sum()

Year      0
Age       0
Pos1_C    0
Pos1_F    0
Pos1_G    0
         ..
Tm_TOT    0
Tm_UTA    0
Tm_VAN    0
Tm_WAS    0
Tm_WSB    0
Length: 485, dtype: int64

In [132]:
# Import sk-learn Libraries

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [133]:
# Linear regression

X_params = possible_X
y_params = possible_y

agg_season_stats_X = agg_season_stats.drop(columns=possible_y)

In [134]:
for param in y_params:
    agg_season_stats_y = agg_season_stats[param] * 100
    X_train, X_test, y_train, y_test = train_test_split(agg_season_stats_X, agg_season_stats_y, test_size=0.3, random_state=42)

    lasso = Lasso(alpha=500)

    lasso.fit(X_train, y_train)
    print(f'Train data R2 score for {param}: {lasso.score(X_train, y_train)}')
    print(f'Test data R2 score for {param}: {lasso.score(X_test, y_test)}')
    print('---------------------------------------------------------')

# We can see that OWS & FG% don't depend much on height & weight, while DWS slightly depends on height & weight

Train data R2 score for DWS: 0.0
Test data R2 score for DWS: -0.0005811445148837802
---------------------------------------------------------
Train data R2 score for PTS: 0.026581909720896912
Test data R2 score for PTS: 0.02796069441682736
---------------------------------------------------------
Train data R2 score for PER: 0.0
Test data R2 score for PER: -0.00010928406657306766
---------------------------------------------------------


In [135]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()

In [136]:
# XGBoost
for param in y_params:
    agg_season_stats_y = agg_season_stats[param] * 100
    X_train, X_test, y_train, y_test = train_test_split(agg_season_stats_X, agg_season_stats_y, test_size=0.3, random_state=42)
    y_train = lbl.fit_transform(y_train)
    y_test = lbl.fit_transform(y_test)
    xgb = XGBClassifier(n_estimators=100, learning_rate=0.3, eval_metric='mae')

    xgb.fit(X_train, y_train)
    print(f'Train data R2 score for {param}: {xgb.score(X_train, y_train)}')
    print(f'Test data R2 score for {param}: {xgb.score(X_test, y_test)}')
    print('---------------------------------------------------------')

Train data R2 score for DWS: 0.3773495404684257
Test data R2 score for DWS: 0.07208079690094078
---------------------------------------------------------
Train data R2 score for PTS: 0.9023421286688408
Test data R2 score for PTS: 0.004150525733259546
---------------------------------------------------------
Train data R2 score for PER: 0.5756300029647199
Test data R2 score for PER: 0.004150525733259546
---------------------------------------------------------
